In [1]:
'''
Code to perform evaluation on 3DFM_BCPop dataset
'''

import os
os.chdir("..")

import numpy as np
from copy import deepcopy
import itertools
import torch
import tifffile

import cv2

from scipy.ndimage import uniform_filter

import data
from data.sampler.fmBCPop3D_sampler import FMBCPop3DSampler
from options.options import Options
from util.util import *
from metric_funcs import *

from models.biospade_model import BioSPADEModel
from models.segment_model import SegmentModel

import models.networks as networks

import matplotlib.pyplot as plt

name = 'FM3D_BCPop' # Name of experiments
pxs = 512 # xy size of patch to sample

# Generate fake images
def forward(vox, powers, frames, z_pos, is_cycle=False):
    n_samples = vox.shape[1]
    in_Dslices = vox.shape[-3]-org_opt.in_Gslices+1
    in_Dslices_org = 4

    fake = torch.zeros([len(powers), n_samples, 1, in_Dslices, *vox.shape[-2:]])
    fake_mu = torch.zeros_like(fake)
    
    for i in range(len(powers)):
        data = {'mesh_semantics': vox,
                'real_semantics': vox,
                'power': n_samples*[powers[i]],
                'frames': n_samples*[frames[i]],
                'z_pos': z_pos}
        
        if is_cycle:
            for z in range(0, in_Dslices-1, in_Dslices_org):
                data['mesh_semantics'] = vox[:,:,z:z+org_opt.in_Gslices+in_Dslices_org-1]
                data['real_semantics'] = data['mesh_semantics']
                data['z_pos'] = np.asarray(z_pos)+z*org_opt.delta_z
                data = tensorize_dict(data)
                model_Cycle.set_input(data)
                fake_ = model_Cycle.forward(True)
                fake[i,:,:,z:z+in_Dslices_org] = fake_[:,None]
                fake = torch.relu(fake)
        else:
            data = tensorize_dict(data)
            fake[i], fake_mu[i] = model_Gan(data,'inference')
    return fake.cpu().numpy(), fake_mu.cpu().numpy()

# Generate testing data
def get_data(loader):
    data_i = next(iter(dataloader))

    real_stack = data_i['real_stack']
    avg_real = data_i['real_stack'].numpy()
    gt = data_i['real_semantics'].numpy()
    z_pos = data_i['z_pos'].numpy()

    mask = np.zeros_like(gt)
    for i in range(mask.shape[1]):
        mask[:,i] = uniform_filter(gt[:,i], 5)>1e-5
        
    real_stack = real_stack.reshape(1,8*9,4,pxs,pxs)
    target = gt[:,:,4:-4].reshape(1,8*9,4,pxs,pxs)

    seg_z_pos = np.zeros((9*len(z_pos[0])))
    for i, z_pos_ in enumerate(z_pos[0]):
        seg_z_pos[i*9:(i+1)*9] = np.asarray(range(9))*.4+z_pos_
        
    return real_stack, avg_real, gt, target, mask, z_pos, seg_z_pos

In [2]:
data_name = 'fmBCPop3D'

org_opt = Options('options/test_options.yaml', data_name)
org_opt.initialize()

org_opt.train_mode = 'SEG'
org_opt.dataset_mode = data_name
org_opt.name = name
org_opt.nThreads = 1

org_opt.batch_size = 1
org_opt.samples_per_instance = 8
org_opt.crop_xy_sz = [pxs,pxs]
org_opt.in_Dslices = 36

style_combs = np.asarray(list(itertools.product(org_opt.powers, org_opt.frames)))

dataloader, dataset = data.create_dataloader(org_opt,'train')
real_stack, avg_real, gt, target, mask, z_pos, _ = get_data(dataloader)

do_seg = False
if do_seg: # Might not be able to fit both loaders on CPU so have to be done seperately
    segloader, seg_dataset = data.create_dataloader(org_opt,'valid')
    seg_real_stack, seg_avg_real, seg_gt, seg_target, seg_mask, seg_z_pos = get_data(segloader)

['/home/cudicm2/Documents/Data/BipolarPopulation/Stack/Stack1_Laser8.tif', '/home/cudicm2/Documents/Data/BipolarPopulation/Stack/Stack2_Laser8.tif', '/home/cudicm2/Documents/Data/BipolarPopulation/Stack/Stack3_Laser8.tif']
stack dataset [FMBCPop3DDataset] of size 16 was created
mesh dataset [FMBCPop3DDataset] of size 16 was created


In [ ]:
import time
text_real = blur_all(avg_real)

start_time = time.time()

real_LBP = calc_LBP(text_real,mask[0,:,4:-4]) # Compute real LBP
print('LBP Done', time.time()-start_time)
real_GLCM = calc_GLCM(text_real,mask[0,:,4:-4]) # Compute real GLCM
print('GLCM Done', time.time()-start_time)
real_COOC = calc_COOC(text_real, mask[0,:,4:-4]>.5) # Compute real COOC
print('COOC Done', time.time()-start_time)

out_sz = (org_opt.number_of_experiments, 1)
pow_out_sz = (org_opt.number_of_experiments+1, len(org_opt.powers))
losses = {'MSE': np.zeros(pow_out_sz),
          'NMSE': np.zeros(pow_out_sz),
          'PSNR': np.zeros(out_sz),
          'JSD': np.zeros(out_sz),
          'GLCM': np.zeros(pow_out_sz),
          'LBP': np.zeros(pow_out_sz),
          'COOC': np.zeros(pow_out_sz),
          'SEG': np.zeros([*out_sz, 3]),}

for exp in range(0,org_opt.number_of_experiments):
    fake_exp = np.zeros(avg_real[:,:,None].shape)

    # Get experiment details
    opt = deepcopy(org_opt)
    exp_str = ''
    if not org_opt.run_all:
        exp_str = '(exp'+str(exp)+')'
        opt.set_experiment(exp)
    opt.name = org_opt.name+exp_str
    opt.paired_translation = False

    # Calculate Segmentation results
    if do_seg:
        for i in range(3):
            opt.seg_instance = i
            model_Seg = SegmentModel(opt)
            model_Seg.eval()
            acc = SEG(model_Seg, seg_real_stack[:,:,None], seg_target, style_combs[:,0], style_combs[:,1], seg_z_pos, batch_sz=16)
            losses['SEG'][exp,:,i] = acc
    
    # Patch too large for GPU. Split 4 times and send smaller subpatches to GPU individually to create entire fake patch
    for i in range(4): 
        gt_ = gt[:,i*org_opt.samples_per_instance//4:(i+1)*org_opt.samples_per_instance//4]
        z_pos_ = z_pos[:,i*org_opt.samples_per_instance//4:(i+1)*org_opt.samples_per_instance//4]

        model_Gan = BioSPADEModel(opt)
        model_Gan.eval()
        fake,_ = forward(gt_, style_combs[:,0], style_combs[:,1], z_pos_)
            
        fake_exp[:,i*org_opt.samples_per_instance//4:(i+1)*org_opt.samples_per_instance//4] = fake
    avg_fake = fake_exp

    avg_fake = avg_fake[:,:,0]
    text_fake = blur_all(avg_fake)
    fake = fake[:,:,0]
    
    # Evaluate
    fake_LBP = calc_LBP(text_fake, mask[0,:,4:-4]) # Use mask as we only want to compute metric around dendrites
    fake_GLCM = calc_GLCM(text_fake, mask[0,:,4:-4]) # Use mask as we only want to compute metric around dendrites
    fake_COOC = calc_COOC(text_fake, mask[0,:,4:-4]>.5) # Use mask as we only want to compute metric around dendrites
    
    losses['MSE'][exp] = MSE(fake_exp[:,:,0]*gt[:,:,4:-4], avg_real[0]*gt[0,:,4:-4]) # Compute MSE
    losses['GLCM'][exp] = MSE(fake_GLCM, real_GLCM, axis=(1,2,3,4)) # Compute MSE of GLCM
    losses['LBP'][exp] = mult_JSDs(fake_LBP, real_LBP) # Compute JSD of LBP
    losses['COOC'][exp] = MSE(fake_COOC, real_COOC, axis=(1,2,3,4,5,6)) # Compute MSE of COOC
    losses['JSD'][exp] = compare_hist(fake, avg_real) # Compute JSD of pixels
    

In [ ]:
# Print Results (Note there will be variance trial to trail since we are sampling patches)

keys = ['MSE', 'LBP', 'GLCM', 'COOC', 'JSD', 'SEG']
scale = {'MSE': 1e3, 'NMSE': 1e1, 'GLCM': 1e8, 'LBP': 1e3, 'PSNR':1e1, 'JSD': 1e2, 'COOC':1e8, 'Frangi':1e5, 'SEG': 100}

ln = ''s
for key in keys:
    ln += key+' & '
print(ln)

for i_loss in range(org_opt.number_of_experiments):
    ln = ''
    for key in keys:
        val = losses[key][i_loss].mean()
        if key == 'SEG':
            val = losses[key][i_loss].mean(0).mean(0)
            var = losses['SEG'][i_loss].mean(0).std(0)
            ln += '%0.3f +/- %0.3f' % (val*scale['SEG'], var*scale['SEG'])
        else:
            ln += '%0.3f ' % (val*scale[key])
    print(ln)